In [1]:
%load_ext autoreload
%autoreload 2

# Word (.docx) to CSV

In [ ]:
"""from utils.translation_helpers import docx_to_csv

word_file = "../data/Parkinson/en/7-1_script_interview_clinique_1 English.docx"
docx_to_csv(word_file, data_directory="../data")"""

# Translation

In [4]:
from transformers import SeamlessM4Tv2ForTextToText, AutoProcessor
import torch
from utils.translation_helpers import translate_by_row_csv, translate_with_chunks

c:\Users\david\anaconda3\envs\seam\lib\site-packages\transformers\deepspeed.py:24: FutureWarning: transformers.deepspeed module is deprecated and will be removed in a future version. Please import deepspeed modules directly from transformers.integrations
  warnings.warn(


In [5]:
print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU found")

True
1
NVIDIA GeForce GTX 1050 Ti


In [6]:
model = SeamlessM4Tv2ForTextToText.from_pretrained("facebook/seamless-m4t-v2-large")
processor = AutoProcessor.from_pretrained("facebook/seamless-m4t-v2-large")
cuda = True

if cuda and torch.cuda.is_available():
    model = model.to("cuda")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
# Check the max position embeddings (equivalent to max token length)
max_length = model.config.max_position_embeddings
print(f"Max token length: {max_length}")

# Check the max new tokens (equivalent to max OUTPUT token length)
model.config.max_new_tokens
print(f"Max new tokens: {model.config.max_new_tokens}")

Max token length: 4096
Max new tokens: 256


### Chuncks translations

In [12]:
from utils.translation_helpers import split_text_into_chunks, translation, parse_and_write_translated_text

In [13]:
source_lang = "fra" # French
target_lang = "eng" # English
#path_file = "../results/Parkinson/fr/7-2_script_interview_clinique_1_13-08-2020.csv" # long file
path_file = "../results/Parkinson/fr/7-1_script_interview_clinique_3_21-08-2020.csv" # short file

In [14]:
import os
def get_files(directory, extensions):
    """Get a list of files in the specified directory and its subdirectories with given extensions."""
    files = []

    for root, dirs, files_in_dir in os.walk(directory):
        for file in files_in_dir:
            if any(file.endswith(ext) for ext in extensions):
                files.append(os.path.join(root, file))
                
    return files

# def to see the max numbers of tokens per row in the csv file
files = get_files("../results/Parkinson/fr", [".csv"])

for file in files:
    max_tokens = 0
    list_tokens = []
    with open(file, "r") as f:
        index = 0
        for line in f:
            index += 1
            tokens = len(line.split())
            list_tokens.append(tokens)
            if tokens > max_tokens:
                max_tokens = tokens
                index_max = index
    print(file," max tokens: ",max_tokens,"at line: ",index_max)
    print("average tokens: ",sum(list_tokens)/len(list_tokens))

../results/Parkinson/fr\7-1_script_interview_clinique_1.csv  max tokens:  104 at line:  227
average tokens:  16.1123595505618
../results/Parkinson/fr\7-1_script_interview_clinique_2_21-08-2020.csv  max tokens:  109 at line:  55
average tokens:  13.65625
../results/Parkinson/fr\7-1_script_interview_clinique_3_21-08-2020.csv  max tokens:  72 at line:  32
average tokens:  19.658536585365855
../results/Parkinson/fr\7-1_script_interview_clinique_3_21-08-2020_fra_to_eng.csv  max tokens:  68 at line:  27
average tokens:  16.470588235294116
../results/Parkinson/fr\7-1_script_interview_clinique_3_21-08-2020_fra_to_eng2022222.csv  max tokens:  70 at line:  20
average tokens:  17.964285714285715
../results/Parkinson/fr\7-1_script_interview_clinique_3_21-08-2020_fra_to_eng_80_tokensmax.csv  max tokens:  27 at line:  4
average tokens:  11.972972972972974
../results/Parkinson/fr\7-1_script_interview_clinique_4_21-08-2020.csv  max tokens:  63 at line:  3
average tokens:  14.181818181818182
../results

In [50]:
max_length = 40
chunks = split_text_into_chunks(path_file, max_length, processor, source_lang)
print(f"Number of chunks: {len(chunks)}\n")

for text in chunks:
    tokens = processor(text, return_tensors="pt", src_lang=source_lang)
    print(f"Number of tokens: {len(tokens['input_ids'][0])}")

chunks

Number of chunks: 47

Number of tokens: 30
Number of tokens: 38
Number of tokens: 38
Number of tokens: 40
Number of tokens: 36
Number of tokens: 40
Number of tokens: 40
Number of tokens: 28
Number of tokens: 32
Number of tokens: 38
Number of tokens: 28
Number of tokens: 30
Number of tokens: 32
Number of tokens: 28
Number of tokens: 36
Number of tokens: 14
Number of tokens: 38
Number of tokens: 36
Number of tokens: 30
Number of tokens: 24
Number of tokens: 32
Number of tokens: 52
Number of tokens: 38
Number of tokens: 36
Number of tokens: 12
Number of tokens: 36
Number of tokens: 38
Number of tokens: 28
Number of tokens: 26
Number of tokens: 26
Number of tokens: 30
Number of tokens: 38
Number of tokens: 38
Number of tokens: 26
Number of tokens: 38
Number of tokens: 34
Number of tokens: 38
Number of tokens: 24
Number of tokens: 24
Number of tokens: 24
Number of tokens: 22
Number of tokens: 26
Number of tokens: 40
Number of tokens: 36
Number of tokens: 34
Number of tokens: 44
Number of to

['[Examinatrice]: Donc, toujours concernant cette sensation de présence dans le dos, ce vieux monsieur en costume gris.',
 "Est-ce que vous avez l'impression, parfois, que c'est cette personne, cette présence essaye d'interagir avec vous, de communiquer avec vous ?",
 '[Patiente]: Pas vraiment, parce qu’ils parlent quand même entre eux, mais très doucement. [Examinatrice]: Le monsieur en gris ?',
 '[Patiente]: Oui mais il parle avec les autres qui sont avec ce monsieur gris. Je le vois, je le vois chaque soir, mais en principe il est tout seul.',
 "[Examinatrice]: Il est tout seul. D'accord. Donc, il n’essaye pas de vous parler ? [Patiente]: Non.",
 "[Examinatrice]: Est-ce que vous avez l'impression qu'il essaie de lire dans vos pensées ou qu'il sait déjà à quoi vous pensez ?",
 '[Patiente]: Euh, je ne sais pas. Des fois, je me demande : Est-ce que qu’ils savent déjà ce qui se passe demain ?',
 '[Examinatrice]: Ça veut dire ? [Patiente]: Oui, parce que. Comment l’expliquer ?

In [51]:
# Translate the chunks
translated_chunks = []
for chunk in chunks:
    translated_text = translation(source_lang, target_lang, chunk, model, processor, True)
    translated_chunks.append(translated_text)

In [52]:
translated_chunks

['[Examiner]: So, still about this feeling of presence in the back, this old gentleman in a gray suit.',
 'Do you sometimes feel that this person, this presence, is trying to interact with you, to communicate with you?',
 "[Patient]: Not really, because they're still talking to each other, but very softly.",
 '[Patient]: Yes, but he talks to the others who are with this gray gentleman. I see him, I see him every night, but in principle he is alone.',
 "[Examiner]: He's all alone. Okay. So he's not trying to talk to you? [Patient]: No.",
 "Do you feel like he's trying to read your mind or does he already know what you're thinking?",
 "[Patient]: Uh, I don't know. Sometimes I wonder: Do they already know what's going to happen tomorrow?",
 '[Examiner]: What does it mean? [Patient]: Yes, because. How do I explain it?',
 'As if he knew what was going to happen tomorrow. Who could predict what was going to happen?',
 "[Patient]: Yes, something like that? Predict? Yes, or maybe he's doing a 

In [49]:
for text in translated_chunks:
    tokens = processor(text, return_tensors="pt", src_lang=source_lang)
    print(f"Number of tokens: {len(tokens['input_ids'][0])}")

Number of tokens: 28
Number of tokens: 28
Number of tokens: 26
Number of tokens: 54
Number of tokens: 20
Number of tokens: 26
Number of tokens: 24
Number of tokens: 42
Number of tokens: 46
Number of tokens: 42
Number of tokens: 42
Number of tokens: 40
Number of tokens: 30
Number of tokens: 24
Number of tokens: 38
Number of tokens: 30
Number of tokens: 50
Number of tokens: 18
Number of tokens: 36
Number of tokens: 28
Number of tokens: 38
Number of tokens: 32
Number of tokens: 28
Number of tokens: 26
Number of tokens: 32
Number of tokens: 48
Number of tokens: 34
Number of tokens: 34
Number of tokens: 50
Number of tokens: 38
Number of tokens: 36
Number of tokens: 16
Number of tokens: 28
Number of tokens: 20
Number of tokens: 30
Number of tokens: 22


In [32]:
# Parse and write the translated text
output_csv = f"{os.path.splitext(path_file)[0]}_{source_lang}_to_{target_lang}_80_tokensmax_sinpuntos.csv"
parse_and_write_translated_text(translated_chunks, output_csv)

### Translation row by row CSV

In [17]:
"""source_lang = "fra" # French
target_lang = "eng" # English
path_file = "../../results/Parkinson/fr/7-1_script_interview_clinique_3_21-08-2020.csv"

translate_by_row_csv(path_file, source_lang, target_lang, model, processor, cuda)"""

'source_lang = "fra" # French\ntarget_lang = "eng" # English\npath_file = "../../results/Parkinson/fr/7-1_script_interview_clinique_3_21-08-2020.csv"\n\ntranslate_by_row_csv(path_file, source_lang, target_lang, model, processor, cuda)'